# Skincare Dossier Generator  
*Creates Word (.docx) and PDF versions of a peer‑reviewed skincare routine dossier, including sunscreen.*  
Generated on 2025-07-07 05:16 UTC

In [ ]:
# ↳ Install one‑time dependencies (runtime resets will need re‑install)
!pip install python-docx reportlab pandas xlsxwriter --quiet

In [ ]:
# OPTIONAL: Save outputs directly to your Google Drive
SAVE_TO_DRIVE = False  # change to True if you want to mount Drive
if SAVE_TO_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    OUTDIR = '/content/drive/MyDrive/skincare_dossier'
else:
    OUTDIR = '/content'
print('Outputs will be saved to:', OUTDIR)

In [ ]:
# ---- DATA BLOCK: tweak concentrations, papers, or schedule here ----
import pandas as pd

summary_bullets = [
    'Primary goals: photo‑protection, pigment control, barrier support, long‑term collagen remodeling.',
    'Actives selected at clinically validated concentrations (see tables).',
    'Layering and frequency optimized to minimize irritation while preserving synergy; broad‑spectrum sunscreen locks in gains.'
]

usage_rows = [
    ['15 % Vitamin C + 0.5 % Ferulic + 1 % E', 'AM (post‑cleanse)', 'Daily', 'Remain daily; dilute first week if tingling', 'Apply before moisturizer/SPF; compatible with niacinamide & azelaic'],
    ['5 % Niacinamide', 'AM and/or PM', 'Daily, up to 2×', 'Stay 1‑2× daily', 'Buffers irritation from acids and retinoids'],
    ['10 % Azelaic Acid', 'PM preferred', '3× week start', 'Advance to nightly as tolerated', 'Use on “buffer” nights away from retinol'],
    ['≈ 7 % Glycolic Acid', 'PM only', '2–3× week non‑consecutive', 'Cap at 3× week', 'Do not use same night as retinol'],
    ['Retinol 0.1 % → 0.3 %', 'PM only', 'Start 0.1 % 1× week', 'Step to 0.3 % and 3× week over 8 wk', 'Skip glycolic on retinol nights; sandwich with moisturizer'],
    ['Broad‑spectrum SPF 30 + (PA+++)', 'AM final step; re‑apply', 'Daily', 'Reapply every 2 h outdoors', 'Apply after moisturizer; essential for pigment control & anti‑ageing']
]
usage_cols = ['Ingredient', 'Best Time', 'Start Frequency', 'How to Advance', 'Layering Notes']

grid_rows = [
    ['Mon', 'Cleanse → Vit C → Niac → SPF', 'Cleanse → Azelaic → Niac'],
    ['Tue', 'Cleanse → Vit C → Niac → SPF', 'Cleanse → Glycolic → Moisturizer'],
    ['Wed', 'Cleanse → Vit C → Niac → SPF', 'Cleanse → Retinol → Moisturizer'],
    ['Thu', 'Same as Mon', 'Cleanse → Azelaic → Niac'],
    ['Fri', 'Same as Mon', 'Cleanse → Glycolic → Moisturizer'],
    ['Sat', 'Same as Mon', 'Cleanse → Retinol → Moisturizer'],
    ['Sun', 'Same as Mon', 'Cleanse → Niac‑rich moisturizer (barrier night)']
]
grid_cols = ['Day', 'AM Routine', 'PM Routine']

# Evidence rows per ingredient (truncated for brevity – extend as desired)
niacinamide_rows = [
    ['Shahmoradi 2013', 'J Res Med Sci', '5 %', 'Double‑blind RCT, 60 acne; 8 wk', 'Efficacy', 'Lesion count ↓; no AEs', 'PMID 23914212'],
    # ... add more rows as needed
]

# similarly define vitc_rows, aza_rows, gly_rows, retinol_rows, sunscreen_rows
vitc_rows, aza_rows, gly_rows, retinol_rows, sunscreen_rows = [], [], [], [], []  # placeholder

evidence_dict = {
    'Niacinamide 5%': niacinamide_rows,
    # add other ingredients when rows filled
}


In [ ]:
from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Table, TableStyle, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
import os, datetime

# --- Helper functions to build Word & PDF ---
def build_word(docx_path):
    doc = Document()
    doc.add_heading('Evidence‑Based Skincare Routine', level=1).alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    doc.add_paragraph('Generated on ' + datetime.datetime.now().strftime('%Y-%m-%d'))
    # Exec summary
    doc.add_heading('Executive Summary', level=2)
    for b in summary_bullets:
        doc.add_paragraph(b, style='List Bullet')
    # Usage map
    doc.add_heading('Usage Map – How & When to Apply', level=2)
    table = doc.add_table(rows=1, cols=len(usage_cols))
    for i, col in enumerate(usage_cols):
        table.rows[0].cells[i].text = col
    for row in usage_rows:
        cells = table.add_row().cells
        for i, val in enumerate(row):
            cells[i].text = val
    doc.add_page_break()
    # ingredient sheets
    for ing, rows in evidence_dict.items():
        doc.add_heading(ing, level=2)
        t = doc.add_table(rows=1, cols=7)
        hdr = ['Study / Year','Journal','Dose','Design & N','Focus','Key Findings','Ref']
        for i, h in enumerate(hdr):
            t.rows[0].cells[i].text = h
        for r in rows:
            cells = t.add_row().cells
            for i, val in enumerate(r):
                cells[i].text = str(val)
        doc.add_page_break()
    doc.save(docx_path)

def build_pdf(pdf_path):
    styles = getSampleStyleSheet()
    elems = [Paragraph('Evidence‑Based Skincare Routine', styles['Title'])]
    elems.append(Paragraph('Generated on ' + datetime.datetime.now().strftime('%Y-%m-%d'), styles['Normal']))
    elems.append(Spacer(1,12))
    # Exec summary
    elems.append(Paragraph('Executive Summary', styles['Heading2']))
    for b in summary_bullets:
        elems.append(Paragraph('• '+b, styles['Normal']))
    elems.append(PageBreak())
    # Build PDF
    pdf = SimpleDocTemplate(pdf_path, pagesize=letter)
    pdf.build(elems)

# --- Output paths ---
os.makedirs(OUTDIR, exist_ok=True)
DOCX_PATH = os.path.join(OUTDIR, 'skincare_dossier.docx')
PDF_PATH  = os.path.join(OUTDIR, 'skincare_dossier.pdf')

build_word(DOCX_PATH)
build_pdf(PDF_PATH)
print('Files saved to', DOCX_PATH, 'and', PDF_PATH)